In [4]:
from utils import generate_SUR, Sampler, DefaultParams
import numpy as np
from numpy.linalg import inv
from scipy.stats import invwishart, invgamma

In [5]:
params = DefaultParams()

In [8]:
class DMC_Sampler(Sampler):
    def __init__(self, SUR_data, params = DefaultParams(), n_samples=10000) -> None:
        super().__init__(SUR_data)
        self.n_samples = n_samples
        self._params = params
        
    def sample(self):
        y1 = self.SUR_data['Y'][:self._params.n]
        y2 = self.SUR_data['Y'][self._params.n:]
        x1 = self.SUR_data['x1']
        x2 = self.SUR_data['x2']
        b1_hat = inv(np.transpose(x1) @ x1) @ np.transpose(x1) @ y1
        a1 = (params.n - 2) / 2
        scale1 = (np.transpose(y1 - x1 @ b1_hat) @ (y1 - x1 @ b1_hat)).item() / 2
        sigma1 = [invgamma.rvs(a=a1, scale=scale1) for _ in range(self.n_samples)]
        mean1 = b1_hat.reshape(-1,)
        b1 = [np.random.multivariate_normal(mean=mean1, cov=inv(np.transpose(x1) @ x1)*sig1).reshape(-1,1) for sig1 in sigma1]
        z2 = [np.concatenate((x2, (y1-x1@b1_ele)), axis=1) for b1_ele in b1]
        b2_hat = [inv(np.transpose(z2_ele) @ z2_ele) @ np.transpose(z2_ele) @ y2 for z2_ele in z2]
        a2 = (params.n - 1) / 2
        scale2 = [(np.transpose(y2 - z2_ele @ b2_hat_ele) @ (y2 - z2_ele @ b2_hat_ele)).item() / 2 
                    for (z2_ele,b2_hat_ele) in zip(z2,b2_hat)]
        sigma2 = [invgamma.rvs(a=a2, scale=scale2_ele) for scale2_ele in scale2]
        b2 = [np.random.multivariate_normal(mean=b2_hat_ele.reshape(-1,), cov=inv(np.transpose(z2_ele) @ z2_ele)*sig2).reshape(-1,1) 
                for (b2_hat_ele,sig2,z2_ele) in zip(b2_hat,sigma2,z2)]
        
        rho = [ele[-1].item() for ele in b2]
        w11 = np.array(sigma1)
        w12 = np.array([rho_ele * sigma1_ele for (rho_ele, sigma1_ele) in zip(rho, sigma1)])
        w22 = np.array([rho_ele * sigma1_ele + sigma2_ele for (rho_ele, sigma1_ele, sigma2_ele) in zip(rho, sigma1, sigma2)])
        Omega = [np.array([[w11_ele, w12_ele], 
                           [w12_ele, w22_ele]]) for (w11_ele, w12_ele, w22_ele) in zip(w11, w12, w22)]
        beta1 = np.array(b1)
        beta2 = np.array(b2)[:,:2,:]
        return beta1, beta2, np.array(Omega)

In [9]:
DMC_sampler = DMC_Sampler(SUR_data=generate_SUR())
beta1, beta2, Omega = DMC_sampler.sample()

In [ ]:
print(f'Mean: {np.mean(beta1, axis=0)};')

Mean: [[ 2.99272022]
 [-2.00489343]];


In [ ]:
print(f'Mean: {np.mean(beta2, axis=0)};')


Mean: [[2.02448857]
 [1.00205464]];


In [ ]:
print(f'Mean: {np.mean(Omega, axis=0)};')

Mean: [[ 0.11265717 -0.06669008]
 [-0.06669008  1.1870755 ]];
